In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl
import tensorflow.keras.backend as K

from tensorflow.keras.applications.vgg16 import VGG16

In [6]:
model = VGG16(weights='imagenet', include_top=True)

- We need to use tf.keras.backened.function to execute one layer at a time. This gives us access to input/output values of each layer. We need the output values of the last layer to pass into the reversing layers. Where we each layer's output values are passed the the next layer up.

In [ ]:
class DLayer():
    """Base Layer for all the Reverse Layers like DConv2D, DMaxPooling2D etc.,
    
    Arguments:
    layer - A trained Layer object like Conv2D, Dense etc., 
    """
    def __init__(self, layer):
        # We don't get the weights in the base class since layers like MaxPooling2D
        # don't have weights.
        self.layer = layer
        
    def normal_pass(self, backend_function, data, learning_phase=0):
        """Computes the normal forward pass of an image through the network and returns the output
        values of that layer.
        
        Arguments:
        data - The output of the previous layer i.e., layer[i+1]
        learning_phase - 
        """
        self.normal_data = backend_function([data, learning_phase])
        return self.normal_data
        
    def reverse_pass(self, backend_function, data, learning_phase=0):
        """Computes the reversing of each layer. For example reversing a Conv2D layer by
        tranposing its weights(feature maps) in vertical and horizontal direction and then
        executing Conv2D on the tranposed weights.
        
        Arguments:
        """
        self.reverse_data = backend_function([data, learning_phase])
        return self.reverse_data
    

class DConv2D(DLayer):
    """
    """
    def __init__(self, layer):

In [ ]:
def visualize_layer(model, image, layer_name, max_activation_only=False):
    """Visualize a single Layer
    """
    pass